In [6]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium
from pandas import json_normalize
from pandas import DataFrame
import itertools
import requests

In [2]:
#Details of Foursquare ID
CLIENT_ID = 'L34METYTMVBNDN4CMT0CNO2LJKBGDTMJGQJDXYCUT0EH2W0W' # your Foursquare ID
CLIENT_SECRET = '4TCL21AGOPUC0XFEXYYGA5XMCKOGBLOROXGNWS3SKQVPKK2C' # your Foursquare Secret
VERSION = '20180604'

In [3]:
City_df=pd.read_html("https://www.connecticut-demographics.com/cities_by_population")[0][:168]
City_df[["City"]]

,City
0,Bridgeport
1,New Haven
2,Stamford
3,Hartford
4,Waterbury
...,...
163,Colebrook
164,Warren
165,Cornwall
166,Canaan


In [191]:

# Get Basic details of restaurants
url = 'https://api.foursquare.com/v2/venues/explore'

#Empty dataframe
Rest_df=DataFrame()
Temp_df=DataFrame()

#Get Langitude and longitude for Hartford
for count,x in DataFrame(City_df["City"]).iterrows():
    address = x.item()+", "+"Connecticut, USA"
    geolocator=Nominatim(user_agent="foursquare_agent")
    location= geolocator.geocode(address)
    if location==None:
        continue
    latitude = location.latitude
    longitude =location.longitude
    longlag= str(latitude) + "," + str(longitude)
    params = dict(
                client_id=CLIENT_ID,
                client_secret=CLIENT_SECRET,
                v='20180323',
                ll= longlag,
                categoryId="4bf58dd8d48988d10f941735" #Indian restaurant Category
                )
    resp = requests.get(url=url, params=params).json()
    Items_df = json_normalize(resp['response']['groups'][0]["items"])
    Temp_df[["venue.id","name","Address","city","state","postalCode","lat","lng"]]=Items_df[["venue.id","venue.name","venue.location.address","venue.location.city","venue.location.state","venue.location.postalCode","venue.location.lat","venue.location.lng"]]
    Rest_df=Rest_df.append(Temp_df)

In [196]:
Rest_df

,venue.id,name,Address,city,state,postalCode,lat,lng
0,597d01202e268057bc8aa4e1,Taj Indian Cuisine,1879 Black Rock Tpke,Fairfield,CT,06825,41.179873,-73.247887
1,4b49437af964a520d76b26e3,Sweet Basil,284 Black Rock Tpke,Fairfield,CT,06825,41.164041,-73.232982
2,58e365b90e0a1e1b3cb16259,Tasty Halal,988 State St,Bridgeport,CT,06605,41.172658,-73.202958
3,4b469ab0f964a520572526e3,Bangalore Restaurant & Bar,1342 Kings Highway Cutoff,Fairfield,CT,06824,41.149116,-73.245885
4,4ee7cfe3e300898eefd50e81,Talha Restaurant/ International Halal Food,NaN,NaN,Connecticut,06606,41.205679,-73.221454
...,...,...,...,...,...,...,...,...
4,4c6820894e429c74958097b6,Indigo Indian Bistro,232 Spencer St,Manchester,CT,06040,41.766080,-72.570779
5,521d1a5e11d276152759f02b,Maharaja Indian,NaN,Mansfield Center,CT,NaN,41.759967,-72.195408
6,5c7c2cf40457b7002c105be6,Kathmandu Kichen & Bar,33 Wilbur Cross Way #103,Storrs Mansfield,CT,06268,41.803684,-72.241249
7,5773f485498e584f9d5f710f,Kathmandu Kitchen,NaN,NaN,Connecticut,06268,41.803684,-72.241249


In [260]:
Rest_df=Rest_df.drop_duplicates()
Rest_df.shape

(326, 10)

In [261]:
Rest_df.to_csv("C:\\Users\\BHUMIKA\\Desktop\\Data Science\\Python for Data Science Repository\\Capstone-Project\\Restaurants.csv")

In [23]:
Rest_df=pd.read_csv("C:\\Users\\BHUMIKA\\Desktop\\Data Science\\Python for Data Science Repository\\Capstone-Project\\Restaurants.csv")

In [24]:
Rest_df.head()

,venue.id,name,Address,city,state,postalCode,lat,lng,Price,Rating
0,597d01202e268057bc8aa4e1,Taj Indian Cuisine,1879 Black Rock Tpke,Fairfield,CT,6825.0,41.179873,-73.247887,Moderate,NaN
1,4b49437af964a520d76b26e3,Sweet Basil,284 Black Rock Tpke,Fairfield,CT,6825.0,41.164041,-73.232982,Moderate,NaN
2,58e365b90e0a1e1b3cb16259,Tasty Halal,988 State St,Bridgeport,CT,6605.0,41.172658,-73.202958,Moderate,NaN
3,4b469ab0f964a520572526e3,Bangalore Restaurant & Bar,1342 Kings Highway Cutoff,Fairfield,CT,6824.0,41.149116,-73.245885,Moderate,6.6
4,4ee7cfe3e300898eefd50e81,Talha Restaurant/ International Halal Food,NaN,NaN,Connecticut,6606.0,41.205679,-73.221454,Moderate,NaN


In [261]:
Rest_df.to_csv("C:\\Users\\BHUMIKA\\Desktop\\Data Science\\Python for Data Science Repository\\Capstone-Project\\Restaurants.csv")

In [34]:
import folium # plotting library
My_map=folium.Map(location=[41.634922,-72.686832],zoom_start=9)
My_map

In [48]:
restaurant=folium.map.FeatureGroup()
Rest_df=Rest_df.dropna(subset=["lat"])
#style the featuregroup
for x,y in Rest_df[["lat","lng"]].iterrows():
    restaurant.add_child(folium.features.CircleMarker
                   (
                    [y["lat"],y["lng"]],
                    radius=5,color="red",
                    fill_color="Red"
                   )
                  )
#add marker
My_map.add_child(restaurant)
My_map

In [95]:
City_Rest_Count= DataFrame(Rest_df["city"].value_counts())
City_Rest_Count

,city
New Haven,18
West Hartford,16
Danbury,15
Middletown,13
Norwalk,12
...,...
Simsbury,1
East Hartford,1
Framingham,1
Wallingford,1


In [96]:
City_Rest_Count.reset_index(inplace=True)
City_Rest_Count.columns=["City","Restaurant Count"]

In [97]:
print(type(City_df))
Merged_population_Restaurant=pd.merge(City_df,City_Rest_Count,on=["City","City"],how="left")

<class 'pandas.core.frame.DataFrame'>


In [98]:
Merged_population_Restaurant

,Rank,City,Population,Restaurant Count
0,1,Bridgeport,145639,4.0
1,2,New Haven,130331,18.0
2,3,Stamford,129309,11.0
3,4,Hartford,123088,6.0
4,5,Waterbury,108276,7.0
...,...,...,...,...
163,164,Colebrook,1484,NaN
164,165,Warren,1457,NaN
165,166,Cornwall,1291,NaN
166,167,Canaan,1143,NaN


In [102]:
Merged_population_Restaurant.replace(np.NaN,0,inplace=True)
print(Merged_population_Restaurant)

    Rank        City Population  Restaurant Count
0      1  Bridgeport     145639               4.0
1      2   New Haven     130331              18.0
2      3    Stamford     129309              11.0
3      4    Hartford     123088               6.0
4      5   Waterbury     108276               7.0
..   ...         ...        ...               ...
163  164   Colebrook       1484               0.0
164  165      Warren       1457               0.0
165  166    Cornwall       1291               0.0
166  167      Canaan       1143               0.0
167  168       Union        894               0.0

[168 rows x 4 columns]


In [ ]:
Merged_population_Restaurant=Merged_population_Restaurant.astype({"Population":"float"})

In [114]:
#Area with zero restaurant
Merged_population_Restaurant.loc[Merged_population_Restaurant["Restaurant Count"]==0]

,Rank,City,Population,Restaurant Count,Ratio
12,13,Bristol,60218.0,0.0,inf
15,16,West Haven,54763.0,0.0,inf
17,18,Stratford,52120.0,0.0,inf
21,22,Enfield,44143.0,0.0,inf
26,27,Trumbull,35976.0,0.0,inf
...,...,...,...,...,...
163,164,Colebrook,1484.0,0.0,inf
164,165,Warren,1457.0,0.0,inf
165,166,Cornwall,1291.0,0.0,inf
166,167,Canaan,1143.0,0.0,inf


In [117]:
Merged_population_Restaurant=Merged_population_Restaurant.loc[Merged_population_Restaurant["Restaurant Count"]!=0]
Merged_population_Restaurant["Ratio"]=Merged_population_Restaurant["Population"]/Merged_population_Restaurant["Restaurant Count"]

In [120]:
Merged_population_Restaurant.sort_values(by=["Ratio"],inplace=True, ascending=False)
Merged_population_Restaurant

,Rank,City,Population,Restaurant Count,Ratio
7,8,New Britain,72767.0,1.0,72767.000000
18,19,East Hartford,50272.0,1.0,50272.000000
20,21,Wallingford,44596.0,1.0,44596.000000
0,1,Bridgeport,145639.0,4.0,36409.750000
39,40,New London,26966.0,1.0,26966.000000
45,46,Simsbury,24799.0,1.0,24799.000000
9,10,Greenwich,62587.0,3.0,20862.333333
3,4,Hartford,123088.0,6.0,20514.666667
52,53,Berlin,20484.0,1.0,20484.000000
11,12,Hamden,60982.0,3.0,20327.333333
